# Colab-First Minimal Trainer

This notebook trains a simple CNN on CIFAR-10 in Google Colab with GPU acceleration.

## Section A: Setup & Mount

In [ ]:
# Minimal deps for run management & logging
!pip -q install pyyaml pandas

In [ ]:
import os, pathlib, yaml, time, shutil, pandas as pd

# --- Base paths (Colab) ---
PROJECT = "/content/ml-colab-agentic"
DATA_DIR = f"{PROJECT}/data"
RUNS_DIR = f"{PROJECT}/runs"

# Optional Drive mount toggle
SAVE_TO_DRIVE = True
DRIVE_DIR = "/content/drive/MyDrive/ml-colab-agentic"  # change if needed

# --- Create a run folder ---
def new_run_id(dataset, model, note=""):
    ts = time.strftime("%Y-%m-%d_%H-%M")
    return "_".join(x for x in [ts, dataset, model, note] if x)

RUN_ID  = new_run_id("cifar10", "simplenet", "amp")
RUN_DIR = f"{RUNS_DIR}/{RUN_ID}"
for sub in [
    "checkpoints",
    "plots/train","plots/val","plots/test","plots/calib",
    "artifacts/train","artifacts/val","artifacts/test","artifacts/calib",
    "cache"
]:
    pathlib.Path(f"{RUN_DIR}/{sub}").mkdir(parents=True, exist_ok=True)

# --- Write cfg.yaml (freeze your config; edit as needed before training) ---
CFG = {
    "seed": 42,
    "epochs": 5,
    "batch_size": 128,
    "lr": 1e-3,
    "dataset": "CIFAR10",
    "data_root": f"{DATA_DIR}/raw",   # downloads land here
    "num_workers": 2,
    "amp": True,
}
with open(f"{RUN_DIR}/cfg.yaml", "w") as f:
    yaml.safe_dump(CFG, f)

# --- Convenience symlinks to "latest" (optional) ---
def safe_symlink(src, dst):
    try:
        if os.path.islink(dst) or os.path.exists(dst):
            os.remove(dst)
        os.symlink(src, dst)
    except Exception:
        # on Drive/Windows symlink may fail → just copy
        if os.path.isdir(src):
            if os.path.exists(dst): shutil.rmtree(dst)
            shutil.copytree(src, dst)
        else:
            shutil.copy2(src, dst)

safe_symlink(RUN_DIR, f"{PROJECT}/outputs")                 # latest outputs → runs/<id>
safe_symlink(f"{RUN_DIR}/checkpoints", f"{PROJECT}/checkpoints")
print("RUN_DIR:", RUN_DIR)

In [ ]:
# Long-form metrics CSV: split, epoch, metric, value
METRICS_CSV = f"{RUN_DIR}/metrics.csv"

def log_metrics(rows):
    """
    rows: list of dicts with keys: split, epoch, metric, value
    example:
      log_metrics([
        {"split":"train","epoch":1,"metric":"loss","value":1.234},
        {"split":"val","epoch":1,"metric":"acc","value":0.78},
      ])
    """
    import pandas as _pd, os as _os
    df = _pd.DataFrame(rows, columns=["split","epoch","metric","value"])
    df.to_csv(METRICS_CSV, mode="a", header=not _os.path.exists(METRICS_CSV), index=False)

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Clone/update the repo into /content/
import os
import subprocess

REPO_PATH = '/content/ml-colab-agentic'
REPO_URL = 'https://github.com/armanfeili/ml-colab-agentic.git'

if os.path.exists(REPO_PATH):
    print(f"{REPO_PATH} already exists. Updating...")
    subprocess.run(['git', '-C', REPO_PATH, 'pull'], check=True)
else:
    print(f"Cloning {REPO_URL}...")
    subprocess.run(['git', 'clone', REPO_URL, REPO_PATH], check=True)

print(f"Repository ready at {REPO_PATH}")

In [ ]:
# Mount Google Drive (optional but recommended)
from google.colab import drive

drive.mount('/content/drive')
print("Google Drive mounted at /content/drive")

In [ ]:
# Install dependencies from requirements.txt
!pip install -q -r /content/ml-colab-agentic/requirements.txt
print("Dependencies installed.")

In [ ]:
# Add repo to path and verify imports
import sys
sys.path.insert(0, REPO_PATH)

from src.utils import (
    set_seed,
    get_device,
    prepare_dataloaders_cifar10,
    SimpleNet,
    train_one_epoch,
    evaluate,
    save_checkpoint,
    append_metrics_csv,
)
import torch
import torch.optim as optim

print("✅ All imports successful!")

## Section B: Config

In [ ]:
# Configuration
CFG = {
    "seed": 42,
    "epochs": 5,
    "batch_size": 128,
    "lr": 1e-3,
    "num_workers": 2,
    "dataset": "CIFAR10",
    "data_root": "/content/data",
    "save_to_drive": True,
    "drive_dir": "/content/drive/MyDrive/ml-outputs",
}

print("Config:")
for key, val in CFG.items():
    print(f"  {key}: {val}")

## Section C: Train

In [ ]:
# Set seed and device
set_seed(CFG["seed"])
device = get_device()
print(f"Device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Prepare dataloaders
print(f"Loading {CFG['dataset']} from {CFG['data_root']}...")
train_dl, test_dl = prepare_dataloaders_cifar10(
    root=CFG["data_root"],
    batch_size=CFG["batch_size"],
    num_workers=CFG["num_workers"],
)
print(f"✅ Train batches: {len(train_dl)}, Test batches: {len(test_dl)}")

In [ ]:
# Initialize model and optimizer
model = SimpleNet(num_classes=10).to(device)
opt = optim.Adam(model.parameters(), lr=CFG["lr"])

print(f"Model initialized on {device}")
print(f"Results will be saved to: {RUN_DIR}")

In [ ]:
# Training loop
from tqdm import tqdm

print(f"\nTraining for {CFG['epochs']} epochs...\n")
best_val_acc = 0.0

for epoch in range(CFG["epochs"]):
    # Train
    train_loss, train_acc = train_one_epoch(model, train_dl, opt, device)
    
    # Evaluate
    val_loss, val_acc = evaluate(model, test_dl, device)
    
    # Save checkpoint
    epoch_ckpt = f"{RUN_DIR}/checkpoints/epoch_{epoch+1:03d}.pt"
    save_checkpoint(model, epoch_ckpt)
    
    # Update best checkpoint if improved
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        import shutil
        shutil.copy2(epoch_ckpt, f"{RUN_DIR}/checkpoints/best.pt")
    
    # Log metrics (long-form)
    log_metrics([
        {"split": "train", "epoch": epoch+1, "metric": "loss", "value": train_loss},
        {"split": "train", "epoch": epoch+1, "metric": "acc", "value": train_acc},
        {"split": "val", "epoch": epoch+1, "metric": "loss", "value": val_loss},
        {"split": "val", "epoch": epoch+1, "metric": "acc", "value": val_acc},
    ])
    
    print(
        f"Epoch {epoch+1}/{CFG['epochs']} | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
    )

print(f"\n✅ Training complete! Best val acc: {best_val_acc:.4f}")
print(f"Results saved to: {RUN_DIR}")

## Section D: Save Artifacts

In [ ]:
# Show run directory contents
import os

print(f"Run directory: {RUN_DIR}\n")

# List all files in the run directory
for root, dirs, files in os.walk(RUN_DIR):
    level = root.replace(RUN_DIR, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        fpath = os.path.join(root, file)
        size = os.path.getsize(fpath) / (1024)  # KB
        print(f'{subindent}{file} ({size:.1f} KB)')

In [ ]:
# Copy entire run to Google Drive (if enabled)
if SAVE_TO_DRIVE:
    import subprocess, shlex
    
    dst = f"{DRIVE_DIR}/runs/{RUN_ID}"
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    
    # Use cp -r to copy entire run directory
    result = subprocess.run(
        shlex.split(f'cp -r "{RUN_DIR}" "{dst}"'),
        capture_output=True,
        text=True,
        check=False
    )
    
    if result.returncode == 0:
        print(f"✅ Copied run to Drive: {dst}")
    else:
        print(f"⚠️  Copy to Drive failed: {result.stderr}")
else:
    print(f"📁 Run saved locally: {RUN_DIR}")
    print(f"   (Set SAVE_TO_DRIVE=True to copy to Google Drive)")

In [ ]:
# Display metrics table
import pandas as pd

if os.path.exists(METRICS_CSV):
    df = pd.read_csv(METRICS_CSV)
    print("Training Metrics (long-form):")
    print(df.to_string(index=False))
    
    # Pivot for easier viewing
    print("\n\nPivoted view:")
    pivot = df.pivot_table(index=['split', 'epoch'], columns='metric', values='value')
    print(pivot)
else:
    print("No metrics file found.")